## Menampilkan data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/PPW/tugas

/content/drive/MyDrive/PPW/tugas


In [4]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.2 MB/s eta 0:00:00


In [16]:
import pandas as pd
import numpy as np

import re

import nltk
from nltk.corpus import stopwords

#stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk

from nltk.tokenize import sent_tokenize

In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
# Path ke file di Google Drive (ubah sesuai lokasimu)
file_path = '/content/drive/MyDrive/PPW/tugas/satu-berita.csv'

# Baca file CSV
df = pd.read_csv(file_path)

df

,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik


## Preprosesing

#### Cleaning

In [19]:
def remove_url(data_berita):
  url = re.compile(r'https?://\S+|www\.S+')
  return url.sub(r'', data_berita)

def remove_html(data_berita):
  html = re.compile(r'<.#?>')
  return html.sub(r'', data_berita)

def remove_emoji(data_berita):
  emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"
      u"\U0001F300-\U0001F5FF"
      u"\U0001F680-\U0001F6FF"
      u"\U0001F1E0-\U0001F1FF""]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', data_berita)

def remove_numbers(data_berita):
  data_berita = re.sub(r'\d+', '', data_berita)
  return data_berita

def remove_symbols(data_berita):
  data_berita = re.sub(r'[^a-zA-Z0-9\s]', '', data_berita)
  return data_berita

# Terapkan fungsi cleansing pada setiap baris di kolom 'Isi Berita'
df['Isi Berita Clean'] = df['Isi Berita'].apply(remove_url)
df['Isi Berita Clean'] = df['Isi Berita Clean'].apply(remove_html)
df['Isi Berita Clean'] = df['Isi Berita Clean'].apply(remove_emoji)
df['Isi Berita Clean'] = df['Isi Berita Clean'].apply(remove_symbols)
df['Isi Berita Clean'] = df['Isi Berita Clean'].apply(remove_numbers)

# Buat dataframe baru dengan dua kolom: 'Isi Berita' dan 'Cleansing'
df_satu_berita = pd.DataFrame({
    'Isi Berita': df['Isi Berita'],
    'Cleansing': df['Isi Berita Clean']
})

# Tampilkan dataframe
df_satu_berita

,Isi Berita,Cleansing
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...


#### Case Folding

In [20]:
def case_folding(text):
    if isinstance(text, str):
      lowercase_text = text.lower()
      return lowercase_text
    else :
      return text

# Terapkan case folding pada kolom 'Cleansing' dari df_clean
df_satu_berita['case folding'] = df_satu_berita['Cleansing'].apply(case_folding)

# Tampilkan dataframe dengan kolom baru 'case folding'
df_satu_berita[['Isi Berita', 'Cleansing', 'case folding']]

,Isi Berita,Cleansing,case folding
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...


#### Tokenisasi

In [21]:
def tokenize(text):
    tokens = text.split()
    return tokens

df_satu_berita['tokenize'] = df_satu_berita['case folding'].apply(tokenize)

df_satu_berita[['Isi Berita', 'Cleansing', 'case folding', 'tokenize']]

,Isi Berita,Cleansing,case folding,tokenize
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...,"[timesindonesia, jakarta, di, bawah, kepemimpi..."


#### Penghapusan Stopword

In [22]:
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
def remove_stopwords(text):
  return [word for word in text if word not in stop_words]

df_satu_berita['stopword removal'] = df_satu_berita['tokenize'].apply(lambda x: ' '.join(remove_stopwords(x)))

df_satu_berita[['Isi Berita', 'Cleansing', 'case folding', 'tokenize', 'stopword removal']]

,Isi Berita,Cleansing,case folding,tokenize,stopword removal
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...,"[timesindonesia, jakarta, di, bawah, kepemimpi...",timesindonesia jakarta kepemimpinan presiden r...


#### Stemming

In [31]:
# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [30]:
# Fungsi stemming
def stemming(text):
    return [stemmer.stem(word) for word in text]

# Terapkan stemming setelah stopwords removal
df_satu_berita['stemming'] = df_satu_berita['stopword removal'].apply(lambda x: ' '.join(stemming(x.split())))

df_satu_berita[['Isi Berita', 'Cleansing', 'case folding', 'tokenize', 'stopword removal', 'stemming']]

,Isi Berita,Cleansing,case folding,tokenize,stopword removal,stemming
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...,"[timesindonesia, jakarta, di, bawah, kepemimpi...",timesindonesia jakarta kepemimpinan presiden r...,timesindonesia jakarta pimpin presiden ri prab...


#### Ekstraksi Kalimat

In [ ]:
def split_into_sentences(text):
    sentences = nltk.tokenize.sent_tokenize(text)
    return sentences

# Terapkan fungsi untuk memisahkan kalimat
kalimat_per_kalimat = split_into_sentences(df_satu_berita['stemming'][0])

# Buat dataframe baru dengan dua kolom: 'no' dan 'kalimat'
df_kalimat = pd.DataFrame({
    'no': [f'kalimat ke-{i+1}' for i in range(len(kalimat_per_kalimat))],
    'kalimat': kalimat_per_kalimat
})

# Tampilkan dataframe
df_kalimat

,no,kalimat
0,kalimat ke-1,timesindonesia jakarta pimpin presiden ri pra...


In [6]:
import pandas as pd
import re
import nltk
nltk.download('punkt')

# Path ke file di Google Drive (ubah sesuai lokasimu)
file_path = '/content/drive/MyDrive/PPW/tugas/satu-berita.csv'

# Baca file CSV
df = pd.read_csv(file_path)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
df

,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik


In [8]:
def remove_url(data_berita):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', data_berita)

def remove_html(data_berita):
    html = re.compile(r'<.*?>')
    return html.sub(r'', data_berita)

def remove_emoji(data_berita):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', data_berita)

def remove_numbers(data_berita):
    return re.sub(r'\d+', '', data_berita)

def remove_symbols(data_berita):
    return re.sub(r'[^a-zA-Z0-9\s]', '', data_berita)

# Terapkan fungsi cleansing pada setiap baris di kolom 'Isi Berita'
df['Isi Berita Clean'] = df['Isi Berita'].apply(remove_url)
df['Isi Berita Clean'] = df['Isi Berita Clean'].apply(remove_html)
df['Isi Berita Clean'] = df['Isi Berita Clean'].apply(remove_emoji)
df['Isi Berita Clean'] = df['Isi Berita Clean'].apply(remove_symbols)
df['Isi Berita Clean'] = df['Isi Berita Clean'].apply(remove_numbers)

# Buat dataframe baru dengan dua kolom: 'Isi Berita' dan 'Cleansing'
df_satu_berita = pd.DataFrame({
    'Isi Berita': df['Isi Berita'],
    'Cleansing': df['Isi Berita Clean']
})



In [15]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def case_folding(text):
    if isinstance(text, str):
        return text.lower()
    else:
        return text

# Terapkan case folding pada kolom 'Cleansing' dari df_clean
df_satu_berita['case folding'] = df_satu_berita['Cleansing'].apply(case_folding)

def tokenize(text):
    return text.split()

df_satu_berita['tokenize'] = df_satu_berita['case folding'].apply(tokenize)

def remove_stopwords(text):
    stop_words = set(nltk.corpus.stopwords.words('indonesian'))
    return [word for word in text if word not in stop_words]

df_satu_berita['stopword removal'] = df_satu_berita['tokenize'].apply(remove_stopwords)

# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming
def stemming(text):
    return [stemmer.stem(word) for word in text]

# Terapkan stemming setelah stopwords removal
df_satu_berita['stemming'] = df_satu_berita['case folding'].apply(lambda x: ' '.join(stemming(x.split())))


# def stemming(text):
#     factory = StemmerFactory()
#     stemmer = factory.create_stemmer()
#     return [stemmer.stem(word) for word in text.split()]

# df_satu_berita['stemming'] = df_satu_berita['stopword removal'].apply(stemming)

def split_into_sentences(text):
    # Menggabungkan kembali kata-kata yang telah di-stemming menjadi satu teks
    text = ' '.join(text)
    # Memisahkan kalimat berdasarkan pola umum
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return sentences

# Terapkan fungsi untuk memisahkan kalimat
kalimat_per_kalimat = split_into_sentences(df_satu_berita['stemming'][0])

# Buat dataframe baru dengan dua kolom: 'no' dan 'kalimat'
df_kalimat = pd.DataFrame({
    'no': [f'kalimat ke-{i+1}' for i in range(len(kalimat_per_kalimat))],
    'kalimat': kalimat_per_kalimat
})

# Tampilkan dataframe
print(df_kalimat)

             no                                            kalimat
0  kalimat ke-1  t i m e s i n d o n e s i a   j a k a r t a   ...


In [41]:
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import nltk

# Path ke file di Google Drive (ubah sesuai lokasimu)
file_path = '/content/drive/MyDrive/PPW/tugas/satu-berita.csv'

# Baca file CSV
df = pd.read_csv(file_path)

# Tambahkan tag <eos> sebagai pelabelan akhir kalimat
def add_eos_tag(text):
    return re.sub(r'\.\s+', ' <eos> ', text)  # Ganti titik (.) dengan <eos>

# Terapkan pelabelan akhir kalimat
df['Isi Berita EOS'] = df['Isi Berita'].apply(add_eos_tag)

# Fungsi preprocessing lainnya
def remove_url(data_berita):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', data_berita)

def remove_html(data_berita):
    html = re.compile(r'<.*?>')
    return html.sub(r'', data_berita)

def remove_emoji(data_berita):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', data_berita)

def remove_numbers(data_berita):
    return re.sub(r'\d+', '', data_berita)

def remove_symbols(data_berita):
    return re.sub(r'[^a-zA-Z0-9\s]', '', data_berita)

# Terapkan cleansing
df['Isi Berita Clean'] = df['Isi Berita EOS'].apply(remove_url)
df['Isi Berita Clean'] = df['Isi Berita Clean'].apply(remove_html)
df['Isi Berita Clean'] = df['Isi Berita Clean'].apply(remove_emoji)
df['Isi Berita Clean'] = df['Isi Berita Clean'].apply(remove_symbols)
df['Isi Berita Clean'] = df['Isi Berita Clean'].apply(remove_numbers)

# Lakukan case folding
def case_folding(text):
    return text.lower()

df['case folding'] = df['Isi Berita Clean'].apply(case_folding)

# Tokenisasi
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenize'] = df['case folding'].apply(tokenize)

# Hapus stopwords
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

df['stopword removal'] = df['tokenize'].apply(remove_stopwords)

# Stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    return [stemmer.stem(word) for word in text]

df['stemming'] = df['stopword removal'].apply(stemming)

# Tampilkan hasil
df_satu_berita = df[['Isi Berita', 'Isi Berita EOS', 'Isi Berita Clean', 'case folding', 'tokenize', 'stopword removal', 'stemming']]
df_satu_berita

# Memecah kalimat setelah stemming menggunakan tag <eos>
def split_sentences(text):
    return text.split(' <eos> ')

df_satu_berita['kalimat_terpecah'] = df['stemming'].apply(lambda x: ' '.join(x)).apply(split_sentences)

# Tampilkan dataframe akhir
df_satu_berita[['Isi Berita', 'kalimat_terpecah']]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-41-8f2054047ffb>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_satu_berita['kalimat_terpecah'] = df['stemming'].apply(lambda x: ' '.join(x)).apply(split_sentences)


,Isi Berita,kalimat_terpecah
0,"TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",[timesindonesia jakarta pimpin presiden ri pra...
